In [1]:
#中時新聞網
#導入套件
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import pandas as pd
import requests
import time as t
import datetime
import os
from urllib.parse import urljoin

# 輸入要搜尋的五檔股票
search_lis =[]
x1 ='長榮'
search_lis.append(x1)
x2 ='陽明'
search_lis.append(x2)
x3 ='萬海'
search_lis.append(x3)
x4 ='群創' 
search_lis.append(x4)
x5 ='友達' 
search_lis.append(x5)

# 先定義能判別日期的函數(只抓取近兩天的新聞)
def distinguish_date(time):    
    time = time.replace('/','-')
    yest = datetime.datetime.now()+ datetime.timedelta(days=-1)
    yest  = yest .strftime("%Y-%m-%d")
    if ( time == datetime.datetime.now().strftime("%Y-%m-%d") ) or ( time == yest  ):
        return True
    else:
        return False

# 開始爬蟲的時間，為了區別每次爬蟲
start_time = datetime.datetime.now().strftime("%Y-%m-%d %H-%M-%S")

# 每檔股票都跑一次
for stock in search_lis:
    title_lis = []
    time_lis = []
    source_lis = []
    author_lis = []
    article_url_lis = []
    
    # 跑每頁的內容
    page = 1
    while 1:    
        url = "https://wantrich.chinatimes.com/"
        url = url + 'search/' + stock + '/' + str(page)
        re = requests.get(url)
        soup = BeautifulSoup(re.text, "html.parser")
        news = soup.find('ul',{'class':'vertical-list'})
        news = news.findAll('div', {'class':'articlebox-compact'})

        for new in news:  
            new_time = new.find('div', {'class':'meta-info'})
            time = new_time.find('span', {'class':'date'}).text

            if distinguish_date(time) == True :
                new_title = new.find("h3",{'class':'title'})
                title = new_title.text       

                article_url = new.find('a')['href']
                article_url = urljoin(url,article_url )
                
                #隔頁爬蟲
                re = requests.get(article_url)
                soup = BeautifulSoup(re.text, "html.parser")
                soup = soup.find('div', {'class':'align-items-center'})
                source = soup.find('div',{'class':'source'}).text
                author = soup.find('div',{'class':'author'}).text
                if author == '':
                    author = '(無特定作者)'
                # 確切新聞發布時間
                time = time + '   ' + soup.find('span',{'class':'hour'}).text  

                #
                title_lis.append(title)
                time_lis.append(time)
                source_lis.append(source)
                author_lis.append(author)
                article_url_lis.append(article_url)
                t.sleep(1)
            elif distinguish_date(time) == False :
                break
    
        if distinguish_date(time) == False :
            break
        page = page + 1
        
    df = pd.DataFrame({"title":title_lis,"time": time_lis ,"author":author_lis ,"link":article_url_lis})
    # 設定檔案儲存路徑，並輸出成excel檔
    path = "D:\python documents"
    path = os.path.join(path,'中時日報',start_time,stock)    
    #print(stock)
    #print(path)
    if not os.path.exists(path):
        os.makedirs(path)
    string = stock + '_News_chinatimes.xlsx'
    df.to_excel(os.path.join(path,string))

        

In [2]:
#經濟日報
#導入套件
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import pandas as pd
import requests
import time as t
import datetime
import os
from urllib.parse import urljoin

# 輸入要搜尋的五檔股票
search_lis =[]
x1 ='長榮'
search_lis.append(x1)
x2 ='陽明'
search_lis.append(x2)
x3 ='萬海'
search_lis.append(x3)
x4 ='群創' 
search_lis.append(x4)
x5 ='友達' 
search_lis.append(x5)

# 先定義能判別日期的函數(只抓取近兩天的新聞)
def distinguish_date(time):    
    time = time.replace('/','-')
    yest = datetime.datetime.now()+ datetime.timedelta(days=-1)
    yest  = yest .strftime("%Y-%m-%d")
    if ( time == datetime.datetime.now().strftime("%Y-%m-%d") ) or ( time == yest  ):
        return True
    else:
        return False

# 開始爬蟲的時間，為了區別每次爬蟲
start_time = datetime.datetime.now().strftime("%Y-%m-%d %H-%M-%S")

# 每檔股票都跑一次
for stock in search_lis:
    title_lis = []
    time_lis = []
    author_lis = []
    article_url_lis = []
    
    # 開始爬蟲
    url = "https://money.udn.com/search/result/1001/"
    url = url + stock
    re = requests.get(url)
    soup = BeautifulSoup(re.text, "html.parser")
    news = soup.find('ul',{'class':'story-list-holder'})
    news = news.findAll('div', {'class':'story__content'})

    for new in news:  
        article_url = new.find('a')['href']
        # 隔頁爬蟲
        re = requests.get(article_url)
        soup = BeautifulSoup(re.text, "html.parser")
        time = soup.find('time',{'class':'article-body__time'}).text 
        # time 格式  '%Y/%m/%d %H:%M:%S' 
        time_spli = time.split(" ")
        if distinguish_date(time_spli[0]) == True :
            title = soup.find('h1',{'id':'story_art_title'}).text
            article__url = article_url
            
            # refine author 資訊 
            author = soup.find('div',{'class':'article-body__info'}).find('span').text
            author_spli = author.split("／")
            author_spli =  author_spli[0]
            author_spli = author_spli.split("/")
            author_spli =  author_spli[0].split(" ")
            #source = author_spli[0]
            author = author_spli[1]
            author = author.replace("記者",'')
            author = author.split("台北")
            author = author[0]
            if author == '':
                author = '(無特定作者)'
            
            #
            title_lis.append(title)
            time_lis.append(time)
            author_lis.append(author)
            article_url_lis.append(article__url)
            t.sleep(1)
        elif distinguish_date(time_spli[0]) == False :
            break
        
    df = pd.DataFrame({"title":title_lis,"time": time_lis ,"author":author_lis ,"link":article_url_lis})
    # 設定檔案儲存路徑，並輸出成excel檔
    path = "D:\python documents"
    path = os.path.join(path,'經濟日報',start_time,stock) 
    #print(stock)
    #print(path)
    if not os.path.exists(path):
        os.makedirs(path)
    string = stock + '_News_economy.xlsx'
    df.to_excel(os.path.join(path,string))

        

In [6]:
# 鉅亨網
# 導入套件
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import pandas as pd
import time as t
import datetime
import os
# 輸入要搜尋的五檔股票
search_lis =[]
x1 ='長榮'
search_lis.append(x1)
x2 ='陽明'
search_lis.append(x2)
x3 ='萬海'
search_lis.append(x3)
x4 ='群創' 
search_lis.append(x4)
x5 ='友達' 
search_lis.append(x5)

# 先定義能判別日期的函數(只抓取近兩天的新聞)
def distinguish_date(time):    
    time = time.replace('/','-')
    time_spl = time.split(' ')
    time =  time_spl[0]
    #yest = datetime.datetime.now()+ datetime.timedelta(days=-1)
    #yest  = yest .strftime("%Y-%m-%d")
    if ( time == datetime.datetime.now().strftime("%Y-%m-%d") ):     #or ( time == yest  ):
        return True
    else:
        return False
    
# 開始爬蟲的時間，為了區別每次爬蟲
start_time = datetime.datetime.now().strftime("%Y-%m-%d %H-%M-%S")

# 開始抓取新聞
# 每檔股票都跑一次
for stock in search_lis:
    title_lis = []
    time_lis = []
    author_lis = []
    article_url_lis = []
    
    my_options = Options()
    # my_options.add_argument(“--headless”)
    my_options.add_argument("--incognito")
    driver = webdriver.Chrome("C:/ProgramData/Anaconda3/Lib/site-packages/selenium/chromedriver.exe",options =my_options)
    #driver = webdriver.Chrome()
    url = 'https://www.cnyes.com/search/news?keyword=' + stock
    driver.get(url)   
    news = driver.find_element_by_xpath('//*[@id="_SearchAll"]/section/div').find_elements_by_tag_name('a'
                                                                                                      )
    for i in range(len(news)):
        article_url = news[i].get_attribute('href')   
        print(article_url)

        
        #news_head = driver.find_element_by_xpath('//*[@id="_SearchAll"]/section/div')
        news_head = news[i].find_element_by_tag_name('h3')
        title = news_head.text
        print(title)        
        #獲取當前視窗控制程式碼
        now_handle = driver.current_window_handle 
        # 點擊文章網址
        news_head.click()                
        #獲取所有視窗控制程式碼
        all_handles = driver.window_handles
        #彈出兩個介面,跳轉到不是主窗體介面
        for handle in all_handles:
            if handle != now_handle:
                driver.switch_to.window(handle)
                try:
                    author = driver.find_element_by_xpath('/html/body/div[1]/div/div/div[2]/main/div[2]/div[2]/span/span').text
                except NoSuchElementException or StaleElementReferenceException or ConnectionRefusedError:
                    author = driver.find_element_by_xpath('/html/body/div[1]/div/div/div[2]/main/div[2]/div/span/span').text

                #author = author.text
                print(author)
                author_spli = author.split(' ')
                if author_spli[0] == author:
                    author = '(無特定作者)'
                author = author_spli[0].replace('鉅亨網記者','')
                print(author)
                time = driver.find_element_by_xpath('/html/body/div[1]/div/div/div[2]/main/div[2]/div[2]/time').text      
                print(time)
                if distinguish_date(time) == True: 
                    author_lis.append(author)
                    title_lis.append(title)
                    article_url_lis.append( article_url)
                    time_lis.append(time)
                    driver.close()
                    driver.switch_to.window(now_handle)
                    t.sleep(1)
                elif distinguish_date(time) == False:
                    driver.quit()
                    break
        if distinguish_date(time) == False:
            break
    df = pd.DataFrame({"title":title_lis,"time": time_lis ,"author":author_lis ,"link":article_url_lis})
    # 設定檔案儲存路徑，並輸出成excel檔
    path = "D:\python documents"
    path = os.path.join(path,'鉅亨網',start_time,stock) 
    print(stock)
    #print(path)
    if not os.path.exists(path):
        os.makedirs(path)
    string = stock + '_News_cnye.xlsx'
    df.to_excel(os.path.join(path,string))    
        
       
        
        
        
    
    
    
    


<ipython-input-6-5a01c47cf96e>:49: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("C:/ProgramData/Anaconda3/Lib/site-packages/selenium/chromedriver.exe",options =my_options)
<ipython-input-6-5a01c47cf96e>:53: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  news = driver.find_element_by_xpath('//*[@id="_SearchAll"]/section/div').find_elements_by_tag_name('a'


https://news.cnyes.com/news/id/4789006
〈航運指數〉SCFI續創高 美西、美東線運價連袂漲上新高


<ipython-input-6-5a01c47cf96e>:75: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  author = driver.find_element_by_xpath('/html/body/div[1]/div/div/div[2]/main/div[2]/div[2]/span/span').text


鉅亨網記者王莞甯 台北
王莞甯
2021/12/17 17:41


<ipython-input-6-5a01c47cf96e>:86: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  time = driver.find_element_by_xpath('/html/body/div[1]/div/div/div[2]/main/div[2]/div[2]/time').text


長榮
https://news.cnyes.com/news/id/4789006
〈航運指數〉SCFI續創高 美西、美東線運價連袂漲上新高
鉅亨網記者王莞甯 台北
王莞甯
2021/12/17 17:41
陽明
https://news.cnyes.com/news/id/4789006
〈航運指數〉SCFI續創高 美西、美東線運價連袂漲上新高
鉅亨網記者王莞甯 台北
王莞甯
2021/12/17 17:41
萬海
https://news.cnyes.com/news/id/4789023
三大法人買超台股57億元 內外資同步敲進中鋼近10萬張
鉅亨網記者郭幸宜 台北
郭幸宜
2021/12/17 17:54
群創
https://news.cnyes.com/news/id/4789305
Fed決策轉鷹、英特爾CEO抵台、大同高層再異動 本周大事回顧
鉅亨網記者魏志豪 台北
魏志豪
2021/12/18 15:04
https://news.cnyes.com/news/id/4789023
三大法人買超台股57億元 內外資同步敲進中鋼近10萬張
鉅亨網記者郭幸宜 台北
郭幸宜
2021/12/17 17:54
友達
